In [2]:
import os
import json
import nltk.data
import pandas as pd
from sentence_transformers import SentenceTransformer, util

In [3]:
# Initializing NLTK sentence tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [10]:
# Loading a suitable SentenceTransformers model 
model = SentenceTransformer('all-mpnet-base-v2')

In [11]:
# Function to split text into chunks of 5 sentences
def split_text_into_chunks(text):
    sentences = tokenizer.tokenize(text)
    return [sentences[i:i + 5] for i in range(0, len(sentences), 5)]

In [12]:
# Directory containing the corpus
corpus_dir = 'Corpus'

In [16]:
# Create a list to store data
data = []

In [17]:
# Looping through each judgment file in the corpus
for filename in os.listdir(corpus_dir):
    if filename.endswith('_Technical.txt'):
        # Extracting passages from the judgment file
        with open(os.path.join(corpus_dir, filename), 'r', encoding='utf-8') as file:
            judgment_text = file.read()
            passages = [passage.strip() for passage in judgment_text.split('__paragraph__') if passage.strip()]
            
            # Combining passages into one and split into chunks
            combined_passage = ' '.join(passages)
            passage_chunks = split_text_into_chunks(combined_passage)

        # Extracting metadata from the corresponding metadata file
        metadata_filename = filename.replace('_Technical.txt', '_Metadata.json')
        metadata_path = os.path.join(corpus_dir, metadata_filename)
        with open(metadata_path, 'r', encoding='utf-8') as metadata_file:
            metadata = json.load(metadata_file)

        # Generate embeddings for each passage chunk and append to the data list
        for passage_chunk in passage_chunks:
            embedding = model.encode(' '.join(passage_chunk), convert_to_tensor=True)
            data.append({'Passage': ' '.join(passage_chunk), 'Metadata': metadata, 'Embedding': embedding})

In [18]:
# Creating a DataFrame from the list of data
df = pd.DataFrame(data)

In [19]:
# Save the data to a CSV file
df.to_csv('passage_metadata_emb.csv', index=False)